In [1]:
import sys
import pandas as pd
sys.path.append("C:/Users/ping/MyDrive/py_files/python/py379/")
from myUtils import pickle_load, pickle_dump
pd.set_option('max_colwidth', 12, 'display.max_columns', 18, 'display.width', 1200, 'display.max_rows',100)
INPUT_DIR = 'C:/Users/ping/OneDrive/Documents/jenn_bb_sales'
path_pickle_dump = f'{INPUT_DIR}/'

In [2]:
df = pickle_load(path_pickle_dump, 'df_grSale_weather')
df.columns

Index(['Date', 'Time', 'Time Zone', 'Category', 'Item', 'Qty', 'Price Point Name', 'SKU', 'Modifiers Applied', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Transaction ID', 'Payment ID', 'Device Name', 'Notes', 'Details', 'Event Type', 'Location', 'Dining Option', 'Customer ID', 'Customer Name', 'Customer Reference ID', 'Unit', 'Count', 'Itemization Type', 'STATION', 'NAME', 'AWND', 'PGTM', 'PRCP', 'TAVG', 'TMAX', 'TMIN', 'WDF2', 'WDF5', 'WSF2', 'WSF5', 'WT01', 'WT02', 'WT08', 'DOW'], dtype='object')

In [31]:
my_col = ['Category', 'Item', 'Qty', 'Gross Sales', 'Discounts', 'Net Sales', 'Tax', 'Event Type',  'Dining Option', 'Customer Name']

In [34]:
# https://datagy.io/pandas-groupby/
for column in my_col:  
  s = df[column]
  grouped = s.groupby(s)
  print(f'{column}')
  print('='*20)
  _dict = {}
  for key in grouped.groups.keys():
    _count = grouped.get_group(key).count()
    _dict[key] = _count
  
  # sort items and counts in the column in reverse order 
  _dict = dict(sorted(_dict.items(), reverse = True, key=lambda item: item[1]))
  for k, v in _dict.items():
      print(f'{k:<45}{v:10,.0f}')
  print('='*20, '\n')

Category
Food                                             17,383
None                                                411
Beverage                                            197
Merch                                               171
RPG                                                 147
Coffee & Tea                                         57

Item
Danish                                            2,193
Cinn Knot                                         1,014
danish                                              978
2x choc                                             863
Choc Croissant                                      793
Choc crx                                            775
crx                                                 733
Twice Baked                                         698
choc crx                                            687
Croissant                                           675
ccc                                                 623
Crx                              

In [5]:
# https://datagy.io/pandas-groupby/
for column in df.columns[3:5]:
  s = df[column]
  grouped = s.groupby(s)
  print(f'{column}')
  print('='*20)
  for key in grouped.groups.keys():
    my_count = grouped.get_group(key).count()
    print(f'{key:<40}:{my_count:>7,.0f}')
  print('='*20, '\n')

Category
Beverage                                :    197
Coffee & Tea                            :     57
Food                                    : 17,383
Merch                                   :    171
None                                    :    411
RPG                                     :    147

Item
2x choc                                 :    863
Almond box                              :      7
Almond cookies                          :     28
Apple Pie Preorder                      :     22
Apple cider                             :     13
Bento Box                               :     35
Bento Box *10% off! Happy earth month!* :     10
Blueberry Muffin                        :     24
Bluerberry Muffin                       :     82
Bouquet and Candy Box Bundle            :     17
Box                                     :    111
Box *10% off! Happy earth month!*       :      5
Brownie                                 :      5
Butterscotch                            :     38
CCC  